In [4]:
#math and linear algebra stuff
import numpy as np

#Math and linear algebra stuff
import scipy.stats as scs

# Deep learning stuff

# Probabilistic stuff
import tensorflow_probability as tfp

#plots
import matplotlib as mpl
mpl.rcParams['figure.figsize'] = (16.0, 9.0)
#mpl.rc('text', usetex = True)
import matplotlib.pyplot as plt
%matplotlib inline

# Normalizing flows for Generative model in Deep Learning

## Introduction

This notebook is somehow a follow up the InformationTheoryOptimization notebook. Many more basic concepts exposed there will be used here. But we will give a short reminder.

## Definitions

### Bayes theorem

Although Bayesian statistics is not really a novel approach, the recent rise of probabilistic programming libraries, availability of a lot of data, and powerful computer to run markov chains, made the tedious task of running bayesian inference algorithms a lot easier.

Let's recall some simple elements about the Bayes theorem that we have been in other notebooks:

For two given random variables, $X$ and $Y$, we can write :
\begin{equation}
    P(\theta|x) = \frac{P(x|\theta)P(\theta)}{P(x)}
\end{equation}


We could use a slightly more formal version, that includes a model $\mathcal{M}$ that we believe is the underlying model for the random variable $x$, parametrized by $\theta$

\begin{equation}
    P(\theta|x,\mathcal{M}) = \frac{P(x|\theta, \mathcal{M})P(\theta|\mathcal{M})}{P(x|\mathcal{M})}
\end{equation}

* Usually, $\theta$ is the random variable that we would like to caracterize. It usually consist in a set of parameters for a given statistical model (mean and variance for a normal distribution for instance).

* On the contrary $x$, called the evidence, is usually derived from a known dataset, or from sampling a real life process. We will see shortly how we can define an empirical distribution from a set of samples. 

* We call $P(x|\theta)$ the likelihood of $x$, given the model distribution  $\theta$. This is the likelihood (or its logarithm) we tried to maximize with various instances of expectation maximization algorithm in the InformationTheory notebook by finding the optimal model parameters $\theta$.
Maximum likelihood had some success in the past for some instance of problems where likelihood or its logarithm were concave and differentiable, and a gradient based methods allowed to find a global maximum.

* We call $P(\theta|x)$ the posterior distribution of $\theta$, given a known (usually empirical distribution of data) $x$. As opposite to the prior, it gives an idea of the probability of the model AFTER some data ($x$) has been seen.

* We call $P(\theta)$ the a-priori distribution for the variable $\theta$. This one can be derived if we have a-priori knowledge on the model parameters $\theta$, it may consist in apriori knowledge of some surrogate parameters that we integrate as a marginal distribution and scale. A prior usually allows us to compute the probability of a given set of parameters $\theta \in \mathbb{R}^n$ that feed the model $\mathcal{M}$.

A priori usually writes:
  \begin{align*}
    y &\rightarrow P(\theta=y) \\
    \mathbb{R}^n &\mapsto [0,1]
  \end{align*}
  
A concrete example is for instance, the use of the framework of random markov field (or Gibbs random field) in a n-dimensional space like an image, were we can consider the pixels as a set of vertices of a graph, and only neighbouring pixels are connected by edges. The probability of a given graph $\mathcal{G} = (\mathcal{V}, \mathcal{E})$ is given by $P(\mathcal{G}) = \alpha e^{-\beta U(\mathcal{G})}$ where $\alpha$ and $\beta$ are normalization factor, and $U(\mathcal{G}) = \sum_{c\in C} V_c(\mathcal(G))$ is a sum of clique potentials $V_c(\mathcal(G))$ over all possible cliques $C$. A very common instance of Gibbs measure is the gaussian-like distribution where, we have, for each neighbouring pair of pixel $p_1-p_2$ (a clique): $V_{p_1-p_2}(\mathcal{G}) = (\mathcal{G}_{p_1}-\mathcal{G}_{p_2})^2$
When one has no apriori on $\theta$, then the highest entropy hypothesis is implicitly used. More pragmatically, we write $P(\theta=y) \sim Uniform(\mathrm{supp} \theta)$ where $\mathrm{supp} y)$ is the support of $\theta$
Notice that, given a discrete process, as long as we received new data, the prior distribution can be taken just as being the previous step posterior.

* Usually $P(x)$ is the empirical distribution that correspond to a given set of outcomes (actual data). It represents the probability of the given outcome of the experiment regardless of the value of the underlying model $\theta$. We can see $P(x)$ as a marginalization of $\theta$ in the joint distribution $P(x,\theta)=P(x|\theta)P(\theta)$ i.e. $P(x) = \int_{\theta} P(x,\theta)$ such that it acts as a constant used to normalize likelihood  / prior product to make sure the result posterior is a distribution.

### Bayesian inference and the sampling task

As briefly seen in the InformationTheory notebook, in the general case, to compute the posterior over the whole support of $\theta$ is extremely complex problem, especially if $\theta$ is a vector from a high dimension space, and the actual distribution of the posterior is very complex (not smooth, many local dense areas, etc...). MCMC types of methods often comes into play here, where one computes $P(\theta=y_0|x)$ for a given $y_0$, then jumps to another point in space $y_1$, with a given probably, dependant on $P(\theta=y|x)$.
Little by little one can get a discrete approximation of the posterior, but it can be very slow.

For instance, as part of tensorflow probability you get the following methods:

In [9]:
print(tfp.mcmc.MetropolisHastings.__doc__)
print("=================================")
print(tfp.mcmc.RandomWalkMetropolis.__doc__)
print("=================================")
print(tfp.mcmc.MetropolisAdjustedLangevinAlgorithm.__doc__)
print("=================================")
print(tfp.mcmc.NoUTurnSampler.__doc__)
print("=================================")
print(tfp.mcmc.ReplicaExchangeMC.__doc__)
print("=================================")
print(tfp.mcmc.HamiltonianMonteCarlo.__doc__)

Runs one step of the Metropolis-Hastings algorithm.

  The [Metropolis-Hastings algorithm](
  https://en.wikipedia.org/wiki/Metropolis%E2%80%93Hastings_algorithm) is a
  Markov chain Monte Carlo (MCMC) technique which uses a proposal distribution
  to eventually sample from a target distribution.

  Note: `inner_kernel.one_step` must return `kernel_results` as a
  `collections.namedtuple` which must:

  - have a `target_log_prob` field,
  - optionally have a `log_acceptance_correction` field, and,
  - have only fields which are `Tensor`-valued.

  The Metropolis-Hastings log acceptance-probability is computed as:

  ```python
  log_accept_ratio = (current_kernel_results.target_log_prob
                      - previous_kernel_results.target_log_prob
                      + current_kernel_results.log_acceptance_correction)
  ```

  If `current_kernel_results.log_acceptance_correction` does not exist, it is
  presumed `0.` (i.e., that the proposal distribution is symmetric).

  The most c

## Kullback Leibler divergence

The Kullback Leibler divergence, also called the relative entropy or discrimination information, can be computed between two distributions, say $x$ and $y$ and writes $ D_{KL}(x\|y) $.
This metric represents the expectation of the difference of the number of bits needed to encode a symbol from the distribution x, weither the encoding is optimal for the distribution x or y.

The general case reads:

\begin{align}
    D_{KL}(x\|y) &= \mathbb{E}_x \left[ log \left( \frac{x}{y} \right) \right]
\end{align}

In the discrete case it reads:

\begin{align}
    D_{KL}(x\|y) &= \sum_{i\in I} p(x_i) log(p(x_i)) -\sum_{i\in I} p(x_i) log(p(y_i)) \\
                 &= -H(x) + H_y(x)
\end{align}
or equivalently
$$
    D_{KL}(x\|y) = \sum_{i\in I} p(x_i) log\left(\frac{p(x_i)}{p(y_i)}\right)
$$


### KL Divergence and maximum likelihood
This chapter was inspired by [this article](http://www.hongliangjie.com/2012/07/12/maximum-likelihood-as-minimize-kl-divergence/)
by Liangjie Hong

#### Definitions
Let's consider $P(x_i|\theta)$ a distribution used to generate (or sample) $N$ points $x_i, i=0,\dots,N-1$

We can define a general distribution model as:
$$
    P_\theta(x) = P(x|\theta)
$$
And, to handle data from real world, we will define the following empirical distribution (also called probability mass function):
$$
    P_D(x) = \sum_{i=0}^{N-1} \frac{1}{N} \delta(x-x_i)
$$

#### KL Divergence between model and data
We can now express the Kullback Leibler divergence between the model distribution and the empirical one:

\begin{align}
    D_{KL} \left( P_D(x)||P_\theta(x) \right) &= \int P_D(x) log(P_D(x)) dx -\int P_D(x) log(P_\theta(x)) dx \\
    &= H(P_D(x)) - \int P_D(x) log(P_\theta(x)) dx
\end{align}

As $H(P_D(x))$ is a constant term, relatd to the empirical distribution of real data, we will call it $\epsilon$, and we may get rid of it later.
Instead we will mainly consider, the $\theta$, ie, the model related term:
$$
   D_{KL} \left( P_D(x)||P_\theta(x) \right) = \epsilon -\langle P_D(x) , log(P_\theta(x)) \rangle_{L^2}
$$

Thanks to the definition of the empirical distribution, we can express this continuous expression as a discrete summation:

\begin{align}
    \langle P_D(x) , log(P_\theta(x)) \rangle_{L^2} &= \int \sum_{i=0}^{N-1} \frac{1}{N} \delta(x-x_i) log(P(x|\theta)) dx \\
    &= \sum_{i=0}^{N-1} \frac{1}{N} log(P(x_i|\theta)) \\
    &= \frac{1}{N} \sum_{i=0}^{N-1} log(P(x_i|\theta))
\end{align}

The last line amounts to the log-likelihood of the dataset. We can then conclude that maximizing the log-likelihood of a dataset, relatively to a set of distribution parameters $\theta$ and a given dataset $x$ is equivalent to minimizing the Kullback Leibler divergence between the empirical distribution and the model distribution.

### Practical KL Divergence between two data sets
We are interested in computing the Kullback Leibler divergence between a model $X$ and a real dataset $Y$, indexed by $i$ where $i$ is the index of a specific event whose occurence probability is $X_i$ according to the model, and $Y_i$ in practice.

As in the definition of the empirical distribution, our datasets must verify:
$$
    \sum_{i=0}^{N-1} X_i = 1
$$

then the expression of the kl divergence is simply:
$$
    D_{KL}(Y\|X) = \sum_{i=0}^{N-1} log \left( \frac{Y_i}{X_i} \right) Y_i
$$

Moreover there is an important condition that must be respected: 
$$
    X_i = 0 \implies Y_i = 0
$$
If so, if the event $k$ has no occurence, we have $X_k=Y_k=0$ and $log \left( \frac{y_k}{x_k} \right) y_k$ has a limit in $0$ which is $0$. Otherwise, Kullback Leibler cannot be computed.

The explanation is pretty straightforward: if an event, or a symbol from $Y$ has no occurence in $X$, it has no reason to be encoded using $X$ probability distribution, then the average number of bits needed to encode a symbol from $Y$ is a nonsense, as some symbols cannot be represented.

## What is variational inference ?

### Variational distribution for the posterior

We are still in the framework of bayes theorem, and we want to compute $p$ our posterior that stands for $p(\theta|x)$.

In variational inference,  you create a variational distribution $q$ over your model parameters $\theta$, that we will call latent variables here. A variational distribution is parametrized by a variational parameter, we will call $\nu$:

$$
 q(\theta|\nu)
$$

The first idea of variational inference, is that we will try to find $\nu$ such that $q(\theta|\nu)$ becomes close to our posterior distribution $p(\theta|x)$.

### KL-divergence for variational/posterior closeness

The closeness of those distribution will be computed using Kullback-Liebler divergence:

\begin{align}
    D_{KL}(q\|p) &= \mathbb{E}_q \left[ log \left( \frac{q(\theta)}{p(\theta|x)} \right) \right]
\end{align}

As already mentionned earlier, it is good to remember that KL-divergence is not symmetric, in terms of semantic, one must keep in mind that the measure is relates to the expectation over $q$ distribution. So basically, rare event with respect to $q$ won't result in a large diverge, even if their occurence is high in $p$ (you will use few bits to represent rare events, that's not a big deal). However, if high probably events from $q$ are rare in the posterior, (hence coded with high number of bits) you will need to code this event (or symbol) many time, with many bits, and the divergence with respect to an optimal coding will be high.

### Jensen's inequality and evidence lower bound (ELBO)

Let's recall, that, for concave functions, over a domain $d$ we have
\begin{align*}
  f\left(\mathbb{E}_d \left[ x \right] \right) \geq \mathbb{E}_d \left[f\left( x \right)\right]
\end{align*}

Let's now rewrite the log probability ouf our evidence, as latent variable marginalization, and introduce our variational distribution $q(\theta)$ with a small trick:
\begin{align*}
  $log(p(x)) &= log \left(\int_{\theta} p(x,\theta) \right) \\
  &= log \left(\int_{\theta} p(x,\theta) \frac{q(\theta)}{q(\theta)} \right) \\
  &= log \left(\mathbb{E}_q \left[ \frac{p(x,\theta)}{q(\theta)} \right] \right) \\
\end{align*}

As $log$ is a nice concave function, we can give a lower bound of the evidence in the previous expression
\begin{align*}
  log(p(x)) &= log \left(\mathbb{E}_q \left[ \frac{p(x,\theta)}{q(\theta)} \right] \right) \\
  log(p(x)) &\geq \mathbb{E}_q \left[ log \left(\frac{p(x,\theta)}{q(\theta)} \right) \right] \\
  log(p(x)) \geq \text{ELBO} &= \mathbb{E}_q \left[ log \left(p(x,\theta)\right) -log\left( q(\theta) \right) \right] \\
  \text{ELBO} &= \underbrace{ \mathbb{E}_q \left[log \left(p(x,\theta)\right)\right]}_{\text{Expectation under} \; q \; \text{of the joint evidence, latent variable probability}} - 
  \underbrace{\mathbb{E}_q \left[log \left( q(\theta) \right) \right]}_{\text{Entropy of our variational distribution }q} \\
\end{align*}

Ok what is the actual interest of this lower bound expression ? We need to understand how this later expression is linked with the KL divergence between $p$ and $q$, that reads:

\begin{align}
    D_{KL}(q\|p) &= \mathbb{E}_q \left[ log \left( \frac{q(\theta)}{p(\theta|x)} \right) \right] \\
    &= \mathbb{E}_q \left[ log \left( q(\theta) \right) \right] - \mathbb{E}_q \left[ log \left( p(\theta|x) \right) \right] \\
        &= \mathbb{E}_q \left[ log \left( q(\theta) \right) \right] - \mathbb{E}_q \left[ log \left( p(\theta,x) \right) \right] + \mathbb{E}_q \left[ log \left( p(x)\right) \right] \\
    &= -\left( \underbrace{\mathbb{E}_q \left[ log \left( p(x,\theta) \right) \right] - \mathbb{E}_q \left[ log \left( q(\theta) \right) \right]}_{\text{= ELBO}}\right) + log \left( p(x)\right)\\
\end{align}

The expectation in the last term $log \left( p(x)\right)$ goes away, as there is no dependance to $q$, it has somehow already been marginalized away. This overall evidence probability is a constant, hence can be discarded for our purpose, which is, finding an optimal surrogate $q(\theta)$ to approximate our posterior $p(\theta|x)$.

### Mean field variational inference

There are plenty of families of distributions that have been studied in the framework of variational inference, exponential families, neural networks, Gaussian processes, ...

Let's assume that the variational distribution factorizes:

\begin{align}
    q(\theta_0, \theta_1, \dots , \theta_{m-1}) = \prod_{j=0}^{m-1} q(\theta_j) \\
\end{align}

That would of course only work if all dimensions are independants. But that's one easy solution to start with a simple variational inference workflow:

1. Choose a model distribution $q$ (normal for instance ?)
2. Derive ELBO expression
3. Perform coordinate ascent on each of the $\theta_i$
4. Eventually repeat until convergence

## Basic introduction to generative paradigm

### Generative interpretation of PCA

To serve as a basic introductory example to generative approach in machine learning/signal processing, we will use the famous PCA example and see how it can be analyzed from a generative perspective.
This example is heavily inspired by Mahdi Karami PhD thesis, chapter 2, later also refered in this document.

Let $z \in \mathbb{R}^d$ be a random vector of observation, PCA can be defined as linear transformation of $z$ into a subspace $\mathbb{R}^{d_0}$ as $r = U^T z$ where $U\in \mathbb{R}^{d\times d_0}$ and $0<d_0\leq d$ such that the columns from $U^T$ that stands for the subspaces selected for the projections capture a maximum of variance with respect to a given set of data.

Let $\left( \mu, \Sigma \right)$ denote the mean and covariance matrices of $z$. As seen in the StatisticalTes notebook, we recall that the solution of PCA is a decorrelation matrix, and explain how it is obtained in next paragraph:

### Recall on PCA
Let $X$ be the matrix containing the centered samples (1 sample per column) each one drawn from the same distribution.
The covariance matrix $\Sigma$ then writes $\Sigma=XX^T$, then the decorrelation matrix $W_D$ should verify,

for decorrelation:

\begin{align*}
  W_D X (W_D X)^T &= W_D X X^T W_D^T\\
  &= W_D \Sigma W_D^T \\
  &= D
\end{align*}

where $W_D \Sigma W_D^T = D$ is a diagonal matrix.
One can use an eigenvalue decomposition of the form $\Sigma = U D U^T$, or equivalently, thanks to positive definitiveness and spectral theorem: $D = U^T \Sigma U$ and make the subsequent identification $W_D=U^T$

By disentangling the linear dependancies in the centered dataset, $U^T$ allows to somehow simplify the resulting distribution. Moreover, if you only keep the $d_0$ eigen vectors associated with the largest eigenvalues, then you project onto a space that where the distribution approximation becomes even simpler (lower dimensional gaussian) while retaining most of the information (information is assumed to be measured in term of variance within this context).
This reduced form can be written:
\begin{align*}
  D_{d_0} = U_{d_0}^T \Sigma U_{d_0}
\end{align*}


### Probabilistic interpretation
Assume a probabilistic model of the form:
\begin{align*}
  \phi \mathtt{\sim} \mathcal{N} \left( \mu_0, D_{d_0}\right), \quad 0 < d_0 \leq d \\
  z|\phi \mathtt{\sim} \mathcal{N} \left( W \phi + \mu_{\epsilon}, \sigma^2 D_{d_0}\right), \quad W\in \mathbb{R}^{d\times d_0} \\
\end{align*}

Let's try to see how the solution for the maximum likelihood for this model would be:

## Transforming distributions with bijectors

### Introduction

The fundamental equation we need here, is known for a very long time, and relates to change of variable, initially used to integrate quantities in physics.
Given an invertible function:$f:\underset{Z}{z} \mapsto \underset{X}{x}$ and a distribution $p_{\theta}(z), \; z \in Z$, the change of variable forumla reads

\begin{align*}
  p_{\theta}'(x) &= p_{\theta}\left( f^{-1}(x) \right) |det \left( J_{f^{-1}} \right)| \\
  p_{\theta}'(x) &= p_{\theta}\left( f^{-1}(x) \right) \frac{1}{|det \left( J_{f}\right)|} \\
\end{align*}

Where $|det \left( J_{f}\right)|$ is the absolute value of the determinant of the Jacobian of $f$.

### Quick recall, what is a Jacobian ?

We first recall that, for a function $f:\underset{\mathbb{R}^n}{x} \mapsto \underset{\mathbb{R}^n}{f(x)}$, we can derive the matrix of partial derivative, called the jacobian matrix:

\begin{align*}
  J=\begin{pmatrix}
    \frac{\partial f_0}{\partial x_0} & \frac{\partial f_0}{\partial x_1} & \dots & \frac{\partial f_0}{\partial x_{n-1}}\\
    \frac{\partial f_1}{\partial x_0} & \frac{\partial f_1}{\partial x_1} & \dots & \frac{\partial f_1}{\partial x_{n-1}}\\
  \vdots & \vdots & \vdots & \vdots \\
  \frac{\partial f_{n-1}}{\partial x_0} & \frac{\partial f_{n-1}}{\partial x_1} & \dots & \frac{\partial f_{n-1}}{\partial x_{n-1}}
\end{pmatrix}
\end{align*}

### Chain rule for transformation composition

As seen in the DifferentiatingPerceptron notebook, for complex functions, that are applied each after another, like in most deep learning models (excepted here we are only interested in bijective layers), the chain rule applies, and we have, for an initial distribution:
\begin{align*}
  p_{\theta}(z), \; z \in Z
\end{align*}
And a chain of bijective transforms:
\begin{align*}
  x = f_{\theta}(z) = f_{K-1} \circ f_{K-2} \circ \dots \circ f_{1} \circ f_{0}(z)
\end{align*}
The following holds
\begin{align*}
  p_{\theta}'(x) &= p_{\theta}(z) \prod_{i=0}^{K-1}\frac{1}{|det \left( J_{f_i}\right)|} \\
  p_{\theta}'(x) &= p_{\theta}\left( f_{\theta}^{-1}(x) \right) \frac{1}{|det \left( J_{f_{\theta}}\right)|} \\
\end{align*}

Throughout this notebook, we will mostly use $z$ as a placeholder for the latent variable, $f_{\theta}$ as the bijective transformation, parametrized by $\theta$ that can generated and observed variable $x$, the later will often have larger dimension, to express images or time series.

Now, let's assume we would like to compute the (log) likelihood of a given sample $x$:
\begin{align*}
  log (p_{\theta}'(x)) &= log(p_{\theta}(z)) - \sum_{i=0}^{K-1} log \left( |det \left( J_{f_i}\right)| \right) \\
\end{align*}

Depending on the invertible set of transform you use, given you have acces to automatic differentiation, this expression of likelihood can be use within an optimization objective in order to learn useful representations encoded by $f_{\theta}$, given a set of training examples $\{x_0, x_1, \dots x_{n-1} \in X\}$.

### VAE vs GANs vs NF

As we have seen previously, VAE use variational inference, which is optimizing for the ELBO, which is a surrogate for the posterior distribution of latent variables given a sample for the encoder. We don't get the exact posterior.

In GAN, there is basically no encoder, hence it is impossible to compute likelihood of latent variable given a sample. We can only train the generator/discriminator in a min/max fashion, with the usual risks of mode collapse, or infinite oscillations.

From a theoretical point of view, NF has very interesting properties, as it allows exact likelihood evaluation, and potentially optimization during training by $log(p_{\theta}(z)) - \sum_{i=0}^{K-1} log \left( |det \left( J_{f_i}\right)| \right)$
Exact posterior inference for a given $x$ throught the invertible transform $z=f_{\theta}^{-1}(x)$

Unfortunately, all those theoretical advantages will come with complex numerical challenges, such as finding transformations with tractable and numerically stable Jacobian determinant computations. One often tries to find transformations which Jacobian is diagonal or triangular.

## Normalizing flows: example of bijective systems in DL

### Non linear independent components estimation (NICE)

#### The triangular trick
In the paper Non Linear independent components estimation (see [here](https://arxiv.org/abs/1410.8516) or [here](https://paperswithcode.com/method/nice) ) the authors use the concept of coupling layer.

Coupling layer subsets input, let say for instance into 2 subsets (by simply projecting onto two complementary and trivial supports) and then apply identity on one part of this subset, and apply more complex transformation $g$ that use both subsets for the second output, with an intermediate transformation $m$ over thr first subset:

* inputs: $z_{0:d-1}$ and $z_{d:D-1}$
* output 1 is identity: $x_{0:d-1} = z_{0:d-1}$
* output 2: $x_{d:D-1} = g(z_{d:D-1},m(z_{0:d-1}))$

If you look carefully, you can see that the Jacobian of said transformation will look like this:

\begin{align*}
  J_f &= \begin{pmatrix}
    \frac{\partial Id_{0:d-1}}{\partial z_{0:d-1}} & \frac{\partial Id_{0:d-1}}{z_{d:D-1}} \\
    \frac{\partial g\left(z_{d:D-1},m(z_{0:d-1}\right)}{\partial z_{0:d-1}} & \frac{ \partial g\left(z_{d:D-1},m(z_{0:d-1}\right)}{\partial z_{d:D-1}} \\
\end{pmatrix} \\
&= \begin{pmatrix}
    Id_{0:d-1} & 0\\
    \frac{\partial g\left(z_{d:D-1},m(z_{0:d-1}\right)}{\partial z_{0:d-1}} & \frac{ \partial g\left(z_{d:D-1},m(z_{0:d-1}\right)}{\partial z_{d:D-1}} \\
\end{pmatrix}
\end{align*}

The problem of having a triangular jacobian then amounts to having the lower right part of previous expression triangular. If we analyse the invertibility requirement, we can see that, Identity being trivially invertible, we should concentrate on $g$.
Now $g$ mainly needs to be invertible with regard to its first argument, given the second, because second argument $m(z_{0:d-1})$ can always be fully inverted thanks to the identity in the other output. In the NICE paper, $g$ is set to be a simple addition, and all the complexity (DNN) is used in the $m$ function.

#### Overcoming additive coupling with scaling matrix
Additive coupling layers like $g$ given as an example, are very limited in the sense that their unit gaussian forbids local expansion or contractions, which drastically reduces the expressivity of such functions.

To start with, given one of our target which is to keep latent space $Z$ distribution to be as simple as possible (gaussian ?) while having the power to be matched with complex multimodal distribution of sample space $X$, we need those complex transformation.
Also in the framework of generative network, we would like high density regions from sample space $X$ to expand to large regions of the latent space $Z$ for expressivity.

The authors show that, this problem can somehow be circumvented by using a diagonal feature scaling matrix $S$ before applying $f$ transformations when decoding from the latent space.
The log likelihood for a given decoding then reads:

\begin{align*}
  log (p_{\theta}'(x)) &= \sum_{i=0}^{D-1} log(p_{\theta}(f^{-1}(x)_i) - log \left( |S_{ii}| \right) \\
\end{align*}

### Beyond dimensionality constraints of bijectors: Real NVP

The problem with bijectors, is that by design, input and output dimensionality should be equal. Although in general, latent space in generative models are expected to be smaller in dimension. This is the problem the authors of [Real NVP](https://arxiv.org/abs/1605.08803) see explanation with code [here](https://paperswithcode.com/method/realnvp) or [here](https://keras.io/examples/generative/real_nvp) tried to address.

In this work, authors tried to use a multiscal approach for images, where each set of latent variable was operating on a different resolution from the sample space.

This allows for instance to drop high resolution elements during inference (TBD)

### Normalizing flows and VAE

Up to now, we have mostly seen how to derive likelihood of samples from a distribution of latent variable (hoping to back propagate the error in likelihood to learn something useful).

We might want to parametrize the approximate posterior $p(x|z)$ in a VAE with a flow to form richer distributions. This is very interesting, as usually in VAE you are restricted to a simple multidimensional gaussian distribution in the latent space, although you could leverage normalizing flow to turn this gaussian encoder output into a more complex distribution before decoding.

![title](data/vae_normalizing_flow.png)
![title](data/vae_normalizing_flow2.png)
courtesy: Ari Seff from [here](https://www.youtube.com/watch?v=i7LjDvsLWCg)

This is the idea of Rezende & all in [Variational Inference with Normalizing Flows](https://arxiv.org/abs/1505.05770)

### Masked autoencoder and masked autoregressive flow for density estimation

Mostly refers to [MADE Masked Autoencoder for Distribution Estimation](https://arxiv.org/abs/1502.03509) see author interview [here](https://www.youtube.com/watch?v=315xKcYX-1w) and [Masked Autoregressive Flow for Density Estimation](https://arxiv.org/abs/1705.07057)

The idea here is to take the likelihood of an output sequence as a product of conditional probability of each element upon its predecessor:

\begin{align*}
  p_{\theta}(x_0, x_1, \dots x_{n-1}) &= \prod_{i=0}^{n-1} p_{\theta}(x_i|x_{0:i})\\
\end{align*}

For each element in the sequence, one can model its distribution as
\begin{align*}
  x_i = \mu_i + z_i \times \sigma_i\\
\end{align*}
with
\begin{align*}
  z_i \mathtt{\sim} \mathcal{N}(0,1) \quad \text{and} \quad\mu_i = f_{\mu}(x_{0:i}) \quad \text{and} \quad \sigma_i = f_{\sigma}(x_{0:i})\\
\end{align*}

See also inverse regressive flows see [improved variational inference with inverse autoregressive flow](https://arxiv.org/abs/1606.04934) and [
Parallel WaveNet: Fast High-Fidelity Speech Synthesis](https://arxiv.org/abs/1711.10433)

## Normalizing flows: main references
Here are a few very important references we might refere to

Basic introduction with a great youtube video:
https://www.youtube.com/watch?v=i7LjDvsLWCg

In [2]:
from IPython.display import IFrame

Lets start with the amazing work of Mahdi Karami. The document is a PhD thesis dedicated to the study of tools for variational inference, normalizing flows, analysis and design of operator with tractable inverse jacobian computation, with additional numerical and convex optimization tools. It also contains applications to latent space representations of complex distributions met in signal/image processing, and much more:

In [6]:
IFrame("doc/NormalizingFlows/Karami_Mahdi_202008_PhD.pdf", width=1200, height=800)

Then lets move on to a very interesting review paper: Normalizing Flows: An Introduction and Review of Current Methods by Ivan Kobyzev, Simon J.D. Prince, and Marcus A. Brubaker.

In [7]:
IFrame("doc/NormalizingFlows/1908.09257.pdf", width=1200, height=800)

Another reference paper, that exposes a lot of important properties of Normalizing flows, and design of methods to leverage them in machine learning / representation learning: Normalizing Flows for Probabilistic Modeling and Inference by George Papamakarios, Eric Nalisnick, Danilo Jimenez Rezende, Shakir Mohamed, Balaji Lakshminarayanan

In [8]:
IFrame("doc/NormalizingFlows/1912.02762.pdf", width=1200, height=800)

Then a paper more targeted to application: Same Same But DifferNet: Semi-Supervised Defect Detection with Normalizing Flows by Marco Rudolph, Bastian Wandt and Bodo Rosenhahn:

In [10]:
IFrame("doc/NormalizingFlows/2008.12577.pdf", width=1200, height=800)